# SETUP

In [ ]:
!git clone 'https://github.com/radiantearth/mlhub-tutorials.git'

Cloning into 'mlhub-tutorials'...
remote: Enumerating objects: 479, done.
remote: Counting objects: 100% (337/337), done.
remote: Compressing objects: 100% (237/237), done.
remote: Total 479 (delta 200), reused 191 (delta 91), pack-reused 142
Receiving objects: 100% (479/479), 39.15 MiB | 28.69 MiB/s, done.
Resolving deltas: 100% (260/260), done.


In [ ]:
!pip install -r '/content/mlhub-tutorials/notebooks/South Africa Crop Types Competition/requirements.txt' -q

In [ ]:
exit(0)

# LIBRARIES

In [ ]:
# Required libraries
import os
import tarfile
import json
from pathlib import Path
from radiant_mlhub.client import _download as download_file

import datetime
import rasterio
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedShuffleSplit

os.environ['MLHUB_API_KEY'] = '96f33e4c9510d0d369d881c6fdefa91502829db09f41e0c92cba8b02fede920b'

# DOWNLOAD DATA

In [ ]:
DOWNLOAD_S1 = False # If you set this to true then the Sentinel-1 data will be downloaded which is not needed in this notebook.

# Select which imagery bands you'd like to download here:
DOWNLOAD_BANDS = {
    'B01': True,
    'B02': True,
    'B03': True,
    'B04': True,
    'B05': True,
    'B06': True,
    'B07': True,
    'B08': True,
    'B8A': True,
    'B09': True,
    'B11': True,
    'B12': True,
    'CLM': False
}

# In this model we will only use Green, Red and NIR bands. You can select to download any number of bands. 
# Our choice relies on the fact that vegetation is most sensitive to these bands. 
# We also donwload the CLM or Cloud Mask layer to exclude cloudy data from the training phase. 
# You can also do a feature selection, and try different combination of bands to see which ones will result in a better accuracy.

In [ ]:
FOLDER_BASE = 'ref_south_africa_crops_competition_v1'

def download_archive(archive_name):
    if os.path.exists(archive_name.replace('.tar.gz', '')):
        return
    
    print(f'Downloading {archive_name} ...')
    download_url = f'https://radiant-mlhub.s3.us-west-2.amazonaws.com/archives/{archive_name}'
    download_file(download_url, '.')
    print(f'Extracting {archive_name} ...')
    with tarfile.open(archive_name) as tfile:
        tfile.extractall()
    os.remove(archive_name)

for split in ['test']:
    # Download the labels
    labels_archive = f'{FOLDER_BASE}_{split}_labels.tar.gz'
    download_archive(labels_archive)
    
    # Download Sentinel-1 data
    if DOWNLOAD_S1:
        s1_archive = f'{FOLDER_BASE}_{split}_source_s1.tar.gz'
        download_archive(s1_archive)
        

    for band, download in DOWNLOAD_BANDS.items():
        if not download:
            continue
        s2_archive = f'{FOLDER_BASE}_{split}_source_s2_{band}.tar.gz'
        download_archive(s2_archive)
        
def resolve_path(base, path):
    return Path(os.path.join(base, path)).resolve()
        
def load_df(collection_id):
    split = collection_id.split('_')[-2]
    collection = json.load(open(f'{collection_id}/collection.json', 'r'))
    rows = []
    item_links = []
    for link in collection['links']:
        if link['rel'] != 'item':
            continue
        item_links.append(link['href'])
        
    for item_link in item_links:
        item_path = f'{collection_id}/{item_link}'
        current_path = os.path.dirname(item_path)
        item = json.load(open(item_path, 'r'))
        tile_id = item['id'].split('_')[-1]
        for asset_key, asset in item['assets'].items():
            rows.append([
                tile_id,
                None,
                None,
                asset_key,
                str(resolve_path(current_path, asset['href']))
            ])
            
        for link in item['links']:
            if link['rel'] != 'source':
                continue
            source_item_id = link['href'].split('/')[-2]
            
            if source_item_id.find('_s1_') > 0 and not DOWNLOAD_S1:
                continue
            elif source_item_id.find('_s1_') > 0:
                for band in ['VV', 'VH']:
                    asset_path = Path(f'{FOLDER_BASE}_{split}_source_s1/{source_item_id}/{band}.tif').resolve()
                    date = '-'.join(source_item_id.split('_')[10:13])
                    
                    rows.append([
                        tile_id,
                        f'{date}T00:00:00Z',
                        's1',
                        band,
                        asset_path
                    ])
                
            if source_item_id.find('_s2_') > 0:
                for band, download in DOWNLOAD_BANDS.items():
                    if not download:
                        continue
                    
                    asset_path = Path(f'{FOLDER_BASE}_{split}_source_s2_{band}/{source_item_id}_{band}.tif').resolve()
                    date = '-'.join(source_item_id.split('_')[10:13])
                    rows.append([
                        tile_id,
                        f'{date}T00:00:00Z',
                        's2',
                        band,
                        asset_path
                    ])
            
    return pd.DataFrame(rows, columns=['tile_id', 'datetime', 'satellite_platform', 'asset', 'file_path'])

In [ ]:
competition_test_df = load_df(f'{FOLDER_BASE}_test_labels')

In [ ]:
import gc
gc.collect()

974

# CREATE DATA

In [ ]:
# This DataFrame lists all types of assets including documentation of the data. 
# In the following, we will use the Sentinel-2 bands as well as labels. 
tile_ids_test = competition_test_df['tile_id'].unique()

In [ ]:
from tqdm import tqdm_notebook
import gc
import warnings
warnings.simplefilter('ignore')

In [ ]:
n_obs = 5

In [ ]:
%%time

competition_test_df['Month'] = pd.to_datetime(competition_test_df['datetime']).dt.month
X = np.empty((0, 12*8),dtype=np.float16)
y = np.empty((0, 1),dtype=np.float16)
field_ids = np.empty((0, 1),np.float16)

for tile_id in tqdm_notebook(tile_ids_test[0:tile_ids_test.shape[0]]):
    tile_df = competition_test_df[competition_test_df['tile_id']==tile_id]

    field_id_src = rasterio.open(tile_df[tile_df['asset']=='field_ids']['file_path'].values[0])
    field_id_array = field_id_src.read(1).flatten()
    nonzeroidx = np.nonzero(field_id_array)[0]
    field_ids = np.append(field_ids, field_id_array[nonzeroidx])

    tile_date_times = tile_df[tile_df['satellite_platform']=='s2']['Month'].unique().tolist()
    X_tile = np.empty((nonzeroidx.shape[0], 0),dtype=np.float16)
    n_X = 0
    for date_time_idx in range(0,len(tile_date_times)):

        month = tile_date_times[date_time_idx]
        # 1. bands arrays :
        b1_src = rasterio.open(tile_df[(tile_df['Month']==month) & (tile_df['asset']=='B01')]['file_path'].values[1])
        b1_array = np.expand_dims(b1_src.read(1).flatten()[nonzeroidx], axis=1)
        
        b2_src = rasterio.open(tile_df[(tile_df['Month']==month) & (tile_df['asset']=='B02')]['file_path'].values[1])
        b2_array = np.expand_dims(b2_src.read(1).flatten()[nonzeroidx], axis=1)
        
        b3_src = rasterio.open(tile_df[(tile_df['Month']==month) & (tile_df['asset']=='B03')]['file_path'].values[1])
        b3_array = np.expand_dims(b3_src.read(1).flatten()[nonzeroidx], axis=1)

        b4_src = rasterio.open(tile_df[(tile_df['Month']==month) & (tile_df['asset']=='B04')]['file_path'].values[1])
        b4_array = np.expand_dims(b4_src.read(1).flatten()[nonzeroidx], axis=1)

        b5_src = rasterio.open(tile_df[(tile_df['Month']==month) & (tile_df['asset']=='B05')]['file_path'].values[1])
        b5_array = np.expand_dims(b5_src.read(1).flatten()[nonzeroidx], axis=1)

        b6_src = rasterio.open(tile_df[(tile_df['Month']==month) & (tile_df['asset']=='B06')]['file_path'].values[1])
        b6_array = np.expand_dims(b6_src.read(1).flatten()[nonzeroidx], axis=1)

        b7_src = rasterio.open(tile_df[(tile_df['Month']==month) & (tile_df['asset']=='B07')]['file_path'].values[1])
        b7_array = np.expand_dims(b7_src.read(1).flatten()[nonzeroidx], axis=1)

        b8_src = rasterio.open(tile_df[(tile_df['Month']==month) & (tile_df['asset']=='B08')]['file_path'].values[1])
        b8_array = np.expand_dims(b8_src.read(1).flatten()[nonzeroidx], axis=1)

        b8A_src = rasterio.open(tile_df[(tile_df['Month']==month) & (tile_df['asset']=='B8A')]['file_path'].values[1])
        b8A_array = np.expand_dims(b8A_src.read(1).flatten()[nonzeroidx], axis=1)

        b9_src = rasterio.open(tile_df[(tile_df['Month']==month) & (tile_df['asset']=='B09')]['file_path'].values[1])
        b9_array = np.expand_dims(b9_src.read(1).flatten()[nonzeroidx], axis=1)

        b11_src = rasterio.open(tile_df[(tile_df['Month']==month) & (tile_df['asset']=='B11')]['file_path'].values[1])
        b11_array = np.expand_dims(b11_src.read(1).flatten()[nonzeroidx], axis=1)

        b12_src = rasterio.open(tile_df[(tile_df['Month']==month) & (tile_df['asset']=='B12')]['file_path'].values[1])
        b12_array = np.expand_dims(b12_src.read(1).flatten()[nonzeroidx], axis=1)

        # X_tile = np.append(X_tile,b2_array,  axis = 1 )
        X_tile = np.append(X_tile,b1_array,  axis = 1 )
        X_tile = np.append(X_tile,b2_array,  axis = 1 )
        X_tile = np.append(X_tile,b3_array,  axis = 1 )
        X_tile = np.append(X_tile,b4_array,  axis = 1 )
        X_tile = np.append(X_tile,b5_array,  axis = 1 )
        X_tile = np.append(X_tile,b6_array,  axis = 1 )
        X_tile = np.append(X_tile,b7_array,  axis = 1 )
        X_tile = np.append(X_tile,b8_array,  axis = 1 )
        X_tile = np.append(X_tile,b8A_array,  axis = 1)
        X_tile = np.append(X_tile,b9_array,  axis = 1 )
        X_tile = np.append(X_tile,b11_array,  axis = 1)
        X_tile = np.append(X_tile,b12_array,  axis = 1)

        # del ndvi,gndvi,wdrvi,ndre,evi,savi ,grndvi,ccci
        del b1_array,b2_array,b3_array,b4_array,b5_array,b6_array,b7_array,b8_array,b8A_array,b9_array,b11_array,b12_array
        del b1_src,b2_src,b3_src,b4_src,b5_src,b6_src,b7_src,b8_src,b8A_src,b9_src,b11_src,b12_src
        # if n_X == n_obs:
        #     break
        
    X = np.append(X, X_tile, axis=0)
    del X_tile , field_id_array , field_id_src 
    gc.collect()

In [ ]:
gc.collect()

# Data

In [ ]:
data = pd.DataFrame(X)
data['field_id'] = field_ids

* **Reduce Memory Usage**

In [ ]:
def reduce_mem_usage(df, verbose=True):
  numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
  start_mem = df.memory_usage().sum() / 1024**2
  for col in df.columns:
      col_type = df[col].dtypes
      if col_type in numerics:
          c_min = df[col].min()
          c_max = df[col].max()
          if str(col_type)[:3] == 'int':
              if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                  df[col] = df[col].astype(np.int8)
              elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                  df[col] = df[col].astype(np.int16)
              elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                  df[col] = df[col].astype(np.int32)
              elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                  df[col] = df[col].astype(np.int64)
          else:
              if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                  df[col] = df[col].astype(np.float16)
              elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                  df[col] = df[col].astype(np.float32)
              else:
                  df[col] = df[col].astype(np.float64)

  end_mem = df.memory_usage().sum() / 1024**2
  print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
  print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

  return df

In [ ]:
data = reduce_mem_usage(data)

In [ ]:
data.head()

In [ ]:
gc.collect()

2013

In [ ]:
# Each field has several pixels in the data. Here our goal is to build a Random Forest (RF) model using the average values
# of the pixels within each field. So, we use `groupby` to take the mean for each field_id
data_grouped = data.groupby('field_id').mean().reset_index()
data_grouped = reduce_mem_usage(data_grouped)
data_grouped

In [ ]:
feat = ["B1","B2","B3","B4","B5","B6","B7","B8","B8A","B9","B11","B12"]
columns = [x + '_Month4' for x in feat] + [x + '_Month5' for x in feat] + \
          [x + '_Month6' for x in feat] + [x + '_Month7' for x in feat] + \
          [x + '_Month8' for x in feat] + [x + '_Month9' for x in feat] + \
          [x + '_Month10' for x in feat] + [x + '_Month11' for x in feat] 
columns = ['field_id'] + columns 

In [ ]:
data_grouped.columns = columns
data_grouped

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data_grouped.to_csv('TestObs2.csv',index=False)
os.makedirs('/content/drive/MyDrive/RadiantEarth',exist_ok=True)
os.makedirs('/content/drive/MyDrive/RadiantEarth/Data',exist_ok=True)
os.makedirs('/content/drive/MyDrive/RadiantEarth/Data/S2Test',exist_ok=True)

!cp 'TestObs2.csv' "/content/drive/MyDrive/RadiantEarth/Data/S2Test/"